In [ ]:
# Reference: https://docs.twelvelabs.io/v1.2/docs/create-indexes
!pip install twelvelabs==0.3.1
!pip show twelvelabs # Check if the package is installed and the version
from twelvelabs import TwelveLabs #Import the library after the installation & kernel restart.
from twelvelabs.models.task import Task

Name: twelvelabs
Version: 0.3.1
Summary: SDK for Twelve Labs API
Home-page: https://github.com/twelvelabs-io/twelvelabs-python
Author: Twelve Labs
Author-email: 
License: UNKNOWN
Location: /usr/local/lib/python3.10/dist-packages
Requires: httpx, pydantic
Required-by: 


In [ ]:
from google.colab import userdata
TL_API_KEY=userdata.get('TL_API_KEY')

In [ ]:
# Create index
import os
from glob import glob
# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "/content/test.mp4"  # In a colab Notebook, just upload the file in the root folder and just give the filename.

client = TwelveLabs(api_key=TL_API_KEY)
engines = [
        {
          "name": "marengo2.6",
          "options": ["visual", "conversation", "text_in_video", "logo"]
        }
  ]


In [ ]:
# Upload videos to index
video_files = glob(VIDEO_PATH)
if(len(video_files)==0):
  raise Exception("Video file was not provided")
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file, language="en")
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")


In [ ]:
from google.colab import userdata
TL_API_KEY=userdata.get('TL_API_KEY')

In [ ]:
# Search by text
#from twelvelabs import TwelveLabs

client = TwelveLabs(api_key=TL_API_KEY)

search_results = client.search.query(
  index_id=index.id,
  query_text="Simple to complex agents",
  options=["visual"]
)

# Utility function to print a specific page
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence} metadata={clip.metadata}"
    )

print_page(search_results.data)

while True:
    try:
        print_page(next(search_results))
    except StopIteration:
        break